In [4]:
import os
import autogen
print(dir(autogen))

['Project', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'project', 'utils']


In [8]:
import asyncio
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

import nest_asyncio

from autogen.agentchat.user_proxy_agent import UserProxyAgent

In [ ]:
#from autogen import AssistantAgent
#from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [ ]:
# Define an asynchronous function that simulates some asynchronous task (e.g., I/O operation)


async def my_asynchronous_function():
    print("Start asynchronous function")
    await asyncio.sleep(2)  # Simulate some asynchronous task (e.g., I/O operation)
    print("End asynchronous function")
    return "input"


# Define a custom class `CustomisedUserProxyAgent` that extends `UserProxyAgent`


class CustomisedUserProxyAgent(UserProxyAgent):
    # Asynchronous function to get human input
    async def a_get_human_input(self, prompt: str) -> str:
        # Call the asynchronous function to get user input asynchronously
        user_input = await my_asynchronous_function()

        return user_input

    # Asynchronous function to receive a message

    async def a_receive(
        self,
        message: Union[Dict, str],
        sender,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        # Call the superclass method to handle message reception asynchronously
        await super().a_receive(message, sender, request_reply, silent)


class CustomisedAssistantAgent(AssistantAgent):
    # Asynchronous function to get human input
    async def a_get_human_input(self, prompt: str) -> str:
        # Call the asynchronous function to get user input asynchronously
        user_input = await my_asynchronous_function()

        return user_input

    # Asynchronous function to receive a message
    async def a_receive(
        self,
        message: Union[Dict, str],
        sender,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        # Call the superclass method to handle message reception asynchronously
        await super().a_receive(message, sender, request_reply, silent)



In [ ]:


def create_llm_config(model, temperature, seed):
    config_list = [
        {
            "model": "<model_name>",
            "api_key": "<api_key>",
        },
    ]

    llm_config = {
        "seed": int(seed),
        "config_list": config_list,
        "temperature": float(temperature),
    }

    return llm_config



In [8]:
import torch
print(torch.__version__)


ImportError: /home/temesgen_gebreabzgi/Tenv/lib/python3.8/site-packages/torch/lib/../../torch.libs/libgomp-4dbbc2f2.so.1.0.0: cannot allocate memory in static TLS block

In [5]:
pip show torch


Name: torch
Version: 2.3.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/temesgen_gebreabzgi/Tenv/lib/python3.8/site-packages
Requires: typing-extensions, jinja2, fsspec, networkx, filelock, sympy
Required-by: ultralytics, ultralytics-thop, torchvision
Note: you may need to restart the kernel to use updated packages.


sdfghj


In [2]:
from autogen import Agent
from PIL import Image
import pytesseract
import cv2
import numpy as np
import torch
from sklearn.cluster import KMeans
from collections import Counter
class ImageAnalysisAgent(Agent):
    def __init__(self):
        super().__init__()
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5x')  # Upgraded model
    def object_identification(self, image: Image.Image):
        img_np = np.array(image)
        results = self.model(img_np)
        objects = results.pandas().xyxy[0].to_dict(orient="records")
        return objects
    def color_identification(self, image: Image.Image, num_colors: int = 5):
        image = np.array(image)
        # Ensure the image has three color channels
        if len(image.shape) == 2 or image.shape[2] == 1:  # Grayscale image
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:  # Image with alpha channel
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        # Flatten the image to shape (num_pixels, 3)
        image = image.reshape((image.shape[0] * image.shape[1], 3))
        # Use KMeans to find the dominant colors
        clt = KMeans(n_clusters=num_colors)
        clt.fit(image)
        # Get the colors and their frequency
        hist = self._centroid_histogram(clt)
        colors = clt.cluster_centers_
        # Convert to a list of RGB tuples
        color_info = []
        for (percent, color) in zip(hist, colors):
            color_info.append({
                'color': color.astype(int).tolist(),
                'percentage': percent
            })
        return color_info
    def _centroid_histogram(self, clt):
        # Create a histogram based on the number of pixels assigned to each cluster
        num_labels = np.arange(0, len(np.unique(clt.labels_)) + 1)
        (hist, _) = np.histogram(clt.labels_, bins=num_labels)
        # Normalize the histogram, so that it sums to one
        hist = hist.astype("float")
        hist /= hist.sum()
        return hist
    def position_extraction(self, image: Image.Image):
        img_np = np.array(image)
        results = self.model(img_np)
        bboxes = results.xyxy[0].numpy()
        positions = []
        for bbox in bboxes:
            x_min, y_min, x_max, y_max, confidence, class_id = bbox[:6]
            positions.append({
                'class_id': int(class_id),
                'confidence': float(confidence),
                'x_min': float(x_min),
                'y_min': float(y_min),
                'x_max': float(x_max),
                'y_max': float(y_max),
                'width': float(x_max - x_min),
                'height': float(y_max - y_min)
            })
        return positions
    def character_recognition(self, image: Image.Image):
        return pytesseract.image_to_string(image)


ImportError: /home/temesgen_gebreabzgi/Tenv/lib/python3.8/site-packages/torch/lib/../../torch.libs/libgomp-4dbbc2f2.so.1.0.0: cannot allocate memory in static TLS block